# Imports

In [3]:
# from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns
import env
import os

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

In [2]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{env.user}:{env.password}@{env.host}/{db}'

In [6]:

def get_cohort():

    if os.path.isfile('cohorts.csv'):
        return pd.read_csv('cohorts.csv', index_col=0)

    else:
        query = '''
        SELECT *
        FROM cohorts;
        '''        
        df= pd.read_sql(query, get_connection('curriculum_logs'))
        df.to_csv('cohorts.csv')
    return df


In [7]:
cohorts = get_cohort()

In [17]:
cohorts = cohorts.rename(columns={'id':'cohort'})

In [11]:
# Import .txt file and convert it to a DataFrame object
df = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None, 
                   names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])

In [19]:
df = pd.merge(df, cohorts, on='cohort')

In [20]:
df.date = pd.to_datetime(df.date)
df = df.set_index('date')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 801476 entries, 2018-01-26 to 2021-04-21
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        801476 non-null  object 
 1   page        801475 non-null  object 
 2   id          801476 non-null  int64  
 3   cohort      801476 non-null  float64
 4   ip          801476 non-null  object 
 5   name        801476 non-null  object 
 6   slack       801476 non-null  object 
 7   start_date  801476 non-null  object 
 8   end_date    801476 non-null  object 
 9   created_at  801476 non-null  object 
 10  updated_at  801476 non-null  object 
 11  deleted_at  0 non-null       object 
 12  program_id  801476 non-null  int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 85.6+ MB


# 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
* `web_dev` : javascript-i
* `data_science` : classification overview

In [22]:
df = df[df.page != '/']

In [34]:
df.program_id = np.where(df.program_id == 3, 'data_science', 'web_dev')

In [49]:
df[df.program_id == 'web_dev']['page'].value_counts()

javascript-i                18193
toc                         17580
search/search_index.json    15331
java-iii                    13162
html-css                    13111
                            ...  
query/personal-site             1
register%20t                    1
quiz115                         1
quic/115                        1
extra                           1
Name: page, Length: 2053, dtype: int64

In [50]:
df[df.program_id == 'data_science']['page'].value_counts()

search/search_index.json                    2203
classification/overview                     1785
1-fundamentals/modern-data-scientist.jpg    1655
1-fundamentals/AI-ML-DL-timeline.jpg        1651
1-fundamentals/1.1-intro-to-data-science    1633
                                            ... 
itc-ml                                         1
python/custom-sorting-functions                1
imports                                        1
5-stats/4.2-compare-means/null                 1
java-ii/object-oriented-programming            1
Name: page, Length: 681, dtype: int64

# 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [188]:
for program in df.program_id.unique():
    print(program)
    #print(df[df.program_id == program].page.value_counts().sort_values(ascending=False).head(50))
    print(df[(df.program_id == program)].page.value_counts().loc[lambda x : x>100].sort_values(ascending=False).tail(50))
    print('---')

web_dev
appendix/further-reading/html-css/flexbox-additional-concepts    168
prework/cli                                                      167
content/php_i/intro-via-interactive-shell                        162
content/laravel/intro/application-structure.html                 161
prework/cli/01-intro                                             160
appendix/professional-development/job-search-workshop            160
content/javascript                                               159
appendix/extra-challenges/locations                              156
content/php_i/gitbook/images/favicon.ico                         154
content/git                                                      152
appendix/extra-challenges/sales-tax                              151
appendix/extra-challenges/pdf-parser                             149
slides/passwords                                                 147
3-sql/database-design                                            142
content/laravel/intro/gitb

In [187]:
for cohort in df.name.unique():
    print(cohort)
    print(df[df.name == cohort].page.value_counts().sort_values(ascending=False).head(10))
    print('---')

Hampton
java-iii                 57
appendix                 55
java-ii                  46
java-i                   46
spring                   43
javascript-i             36
javascript-ii            31
java-iii/servlets        30
mysql                    30
java-iii/jsp-and-jstl    27
Name: page, dtype: int64
---
Teddy
spring                      744
java-iii                    732
mysql                       638
javascript-i                615
mkdocs/search_index.json    595
java-i                      522
jquery                      512
java-ii                     509
appendix                    505
javascript-ii               487
Name: page, dtype: int64
---
Sequoia
spring                                         364
spring/fundamentals/views                      207
javascript-i                                   202
spring/fundamentals/controllers                190
spring/fundamentals/repositories               172
appendix                                       170
mkdocs/search_

# 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
* Student ID's 832, 278, 679, 956, 812, 388, 572, 889, 968, and 963 all access curriculum 15 or less time durings their cohorts


In [178]:
df.loc[df['id'].isin([832, 278, 679, 956, 812, 388, 487, 889, 968, 963])].groupby('id')[['name', 'program_id', 'start_date', 'end_date']].max()

,name,program_id,start_date,end_date
id,,,,
278,Voyageurs,web_dev,2018-05-29,2018-10-11
388,Andromeda,web_dev,2019-03-18,2019-07-30
487,Bayes,data_science,2019-08-19,2020-01-30
679,Darden,data_science,2020-07-13,2021-01-12
812,Hyperion,web_dev,2020-05-26,2020-11-10
832,Jupiter,web_dev,2020-09-21,2021-03-30
889,Marco,web_dev,2021-01-25,2021-07-19
956,Oberon,web_dev,2021-04-12,2021-10-01
963,Oberon,web_dev,2021-04-12,2021-10-01


In [135]:
df['in_session_access'] = np.where((df.index < df.end_date) & (df.index > df.start_date), 1, 0)

In [177]:
df[df['in_session_access'] == 1].groupby('id').sum().sort_values(by='in_session_access').head(15)

,cohort,post_grad_access,in_session_access
id,,,
832,124.0,0,2
278,72.0,0,3
679,177.0,0,3
956,556.0,0,4
812,406.0,0,7
388,217.0,0,7
572,583.0,0,11
889,1890.0,0,14
968,2085.0,0,15


# 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?


# 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?


# 6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
* Did not detect any post graduation access from Data Science students
* For Web Dev students Top 5 most accessed were: javascript-i, spring, search, html-css, and java-iii 

In [99]:
# nmake a loop that checks each student, and for the days they are actgive comapres the amount of page views
# they have to the population average and checks if they in the bottom %25 of page views
df[['start_date', 'end_date']] = df[['start_date', 'end_date']].apply(pd.to_datetime)

In [107]:
df['post_grad_access'] = np.where(df.index > df.end_date, 1, 0)

In [181]:
df.groupby(['program_id', 'page'])['post_grad_access'].sum().sort_values(ascending=False).head(60)

program_id  page                                                                     
web_dev     javascript-i                                                                 4965
            spring                                                                       4261
            search/search_index.json                                                     3681
            html-css                                                                     3678
            java-iii                                                                     3537
            java-ii                                                                      3439
            java-i                                                                       3123
            appendix                                                                     3071
            javascript-ii                                                                2978
            mysql                                                   

# 7. Which lessons are least accessed? 
* Filtered for lessons that were accessed at least once
* Subject matter of list members appears indicative of deprecated pages, and not simply a lack of interest or usage. 
* Top 10 Least Accessed: Introduction to Python, Understand, Evaluate, Database-Design, 12-distributed-ml/6.3-prepare-part-3, sql.mysql-overview, tidy-data, classification/explore-old, data-science-pipeline, java-ii/object-oriented-programming


In [89]:
least_accessed = df[(df.program_id == program)].page.value_counts().loc[lambda x : x > 0].sort_values(ascending=False).tail(50)

In [90]:
remove = ['.html','.ico','.svg','.json'] 

pattern = '|'.join(remove)

least_accessed = least_accessed.loc[~least_accessed.index.str.contains(pattern, case=False)]

In [91]:
least_accessed.tail(10)

introduction-to-python                  1
understand                              1
evaluate                                1
database-design                         1
12-distributed-ml/6.3-prepare-part-3    1
sql.mysql-overview                      1
tidy-data                               1
classification/explore-old              1
data-science-pipeline                   1
java-ii/object-oriented-programming     1
Name: page, dtype: int64